In [39]:
# add your stream based IP into the KV260 PYNQ
# This notebooks shows a simple stream based HLS IP,
# you can send data into the HLS IP using Python. The data will be transfered to the PL using stream interface

In [1]:
# To program the FPGA and show the info regarding the IP and the name of top function
from pynq import Overlay
ol = Overlay("./dma_ip_example.bit")
ol.ip_dict

The HLS has the default name from the Vivado project of *example_0*. 

Check *help* for the HLS IP:

In [3]:
ol.example_0?

Type:        DefaultIP
String form: <pynq.overlay.DefaultIP object at 0xffff7aa68340>
File:        /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/overlay.py
Docstring:  
Driver for an IP without a more specific driver

This driver wraps an MMIO device and provides a base class
for more specific drivers written later. It also provides
access to GPIO outputs and interrupts inputs via attributes. More specific
drivers should inherit from `DefaultIP` and include a
`bindto` entry containing all of the IP that the driver
should bind to. Subclasses meeting these requirements will
automatically be registered.

Attributes
----------
mmio : pynq.MMIO
    Underlying MMIO driver for the device
_interrupts : dict
    Subset of the PL.interrupt_pins related to this IP
_gpio : dict
    Subset of the PL.gpio_dict related to this IP


In [4]:
# Create aliases for calling of the overlay and send and receieve channel of the DMA block
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

hls_ip = ol.example_0 

In [5]:
# Check the register map, this is important step to check the status of the HLS IP.

# Note that the HLS IP is not started yet (AP_START=0). You can also see the IP is *idle* (AP_IDLE=1).
# We will start the HLS IP and then start some transfers from the DMA. 
# We could initiate the DMA transfers first if we preferred. The DMA transfers would *stall* until the IP is started. 

hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [6]:
# To start the IP, we can write the bit into the 0x81 address, 
# The writing only need to be done onece

CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

Check the correct bits have been set.

In [8]:
# now we prepare the input data needed to be sent to the HLS IP 
# Note: The space you allowcated and the datatype you use here is important, this should match with the HLS design where you define your stream interface.
#       The sie of the interface will also affect your Vivado Block design, 
#       you may also need to change the width of data there, more specially the DMA block, Stream data width 
from pynq import allocate
import numpy as np

data_size = 5
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(data_size):
    input_buffer[i] = i

In [35]:
# We can define a function that send and receieve the data to and from the DMA
# Note: the Wait function can be very useful, you can check if your transfer had done, 
#.      If the wait function can not complete, stalling may happen, common issue can be wrong input data type or space, no TLAST signal in HLS design
def runKernel():
    dma_send.transfer(input_buffer)
    print("DMA send transfer")
    dma_recv.transfer(output_buffer)
    print("DMA recv transfer")
    dma_send.wait()
    print(" send wait done")
    dma_recv.wait()
    print(" resv wait done")

In [48]:
for i in range(data_size):
    input_buffer[i] = i+10

# Restart the DMA transfer, to see if your design is correct
runKernel()
for i in range(5):
    print(output_buffer[i])

DMA send transfer
DMA recv transfer
 send wait done
 resv wait done
15
16
17
18
19


In [ ]:
runKernel()
for i in range(5):
    # print('0x' + format(output_buffer[i], '02x'))
    print(output_buffer[i])

In [44]:
# You should obtain the similar results such as 
# It shows the HLS IP had started, has autoreset 
# RegisterMap {
#   CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
#   GIER = Register(Enable=0, RESERVED=0),
#   IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
#   IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
# }

hls_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [26]:
dma_send?

Type:           _SDMAChannel
String form:    <pynq.lib.dma._SDMAChannel object at 0xffffa56e27a0>
File:           /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/dma.py
Docstring:     
Drives a single channel of the Xilinx AXI Simple DMA

This driver is designed to be used in conjunction with the
`pynq.allocate()` method of memory allocation. The channel has
main functions `transfer` and `wait` which start and wait for
the transfer to finish respectively. If interrupts are enabled
there is also a `wait_async` coroutine.

This class should not be constructed directly, instead used
through the AxiDMA class.
Init docstring:
Initialize the simple DMA object.

Parameters
----------
mmio : MMIO
    The MMIO controller used for DMA IP.
max_size : int
    Max size of the DMA buffer. Exceeding this will hang the system.
width : int
    Number of bytes for each data.
tx_rx : int
    Set to DMA_TYPE_TX(1) for sending or DMA_TYPE_RX(0) for receiving.
dre : bool
    Data alignment 

In [45]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer-5)))

Arrays are equal: True


## Free all the memory buffers
Don't forget to free the memory buffers to avoid memory leaks! 

In [ ]:
del input_buffer, output_buffer